In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_activity = 'all'
import pandas as pd
train = pd.read_csv('https://classes.soe.ucsc.edu/cmps242/Spring18/hw/hw3/train.csv',encoding='Latin-1')
test = pd.read_csv('https://classes.soe.ucsc.edu/cmps242/Spring18/hw/hw3/test.csv',encoding='Latin-1')

# Preprocess Training Set 

In [3]:
# tokenization
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
filter_sms = [sms for sms in train.sms.values]
word_token = [word_tokenize(sms) for sms in filter_sms]
filter_words = []
for word in word_token:
    filter_words.append([w for w in word if not w in stop_words])
filter_sms = []
for word in filter_words:
    filter_sms.append(' '.join(word))
vectorizer = CountVectorizer()
train_token = vectorizer.fit_transform(filter_sms)

# tfidf transform
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf = True)
tfidf_train_token = transformer.fit_transform(train_token)
tfidf_train_token = tfidf_train_token.toarray()
colname = vectorizer.get_feature_names()
df_token = pd.DataFrame(data=tfidf_train_token,columns=colname)

In [4]:
# Standardizer
from sklearn import preprocessing
standardScaler = preprocessing.StandardScaler(with_mean=False)
standard_df_token = standardScaler.fit_transform(df_token)

In [5]:
# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=1500)
df_token_pca=pca.fit_transform(standard_df_token)

In [6]:
# add bias term
import numpy as np
train_x = np.hstack((np.ones((3000,1)),df_token_pca))

In [7]:
# preprocess the data
np.random.seed(0) # set the seed to 0
w_init = np.random.rand(1501,1)
train_y=pd.get_dummies(train.label.values)['spam'].values
train_y=train_y.astype('float').reshape(len(train_y),1)

# concatenate x and y
trainset = np.hstack((train_x,train_y))

# Gradient Descent
cost function:$$L = -y\log(\sigma(XW))-(1-y)\log(1-\sigma(XW)) + \lambda||W||^2$$
gradient:  $$diag(\sigma(WX) - y)X + \lambda |W|$$
update: $$w = w - \eta \times gradient$$


In [8]:
# likelihood function/logit loss
def l_loss(y_,y):
    if y==1:
        loss = -np.log(y_)
    elif y==0:
        loss = -np.log(1-y_)
    return loss
l_loss = np.vectorize(l_loss)

def logitloss(y_,y):
    loss = l_loss(y_,y)
    where_inf = np.isinf(loss)
    loss[where_inf] = 150
    return loss

def sigmoid(w,x):
    return np.divide(np.exp(np.dot(x,w)),(np.add(1,np.exp(np.dot(x,w)))))

def gradient(y,x,w,lmd):
    #loss = logitloss(sigmoid(w,x),y).reshape(y.shape[0],)
    reg = lmd*w
    reg[0] = 0 # not regularize the bias term
    return np.add(reg,np.dot(np.diag(np.subtract(sigmoid(w,x).reshape(y.shape[0],),y)),x)).mean(axis=0).reshape((1501,1))
    

def gradientDescent(y,w,x,eta,lmd):
    # eta: learning rate
    # lmb: regularization term
    for i in range(1000):
        lossPre = logitloss(sigmoid(w,x),y).sum()
        w = np.subtract(w,np.multiply(eta,gradient(y,x,w,lmd)))
        lossPost = logitloss(sigmoid(w,x),y).sum()
        if (lossPre-lossPost)/lossPre<0.05:
            break
    return w
        

# vectorize functions
logitloss = np.vectorize(logitloss)

#w_opt = gradientDescent(train_y, w_init, train_x, 0.01, 0.1)
#w_opt.shape



# Cross Validation

In [15]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
lambda_candi = [0,0.01,0.1,1,10,100]
error_sum = []
i = 1
for lmd in lambda_candi:
    error = []
    for train_index, test_index in kf.split(train_x):
        w_init = np.random.rand(1501,1)
        X_train, y_train = train_x[train_index], train_y[train_index]
        X_test, y_test = train_x[test_index], train_y[test_index]
        w_opt = gradientDescent(y_train,w_init,X_train,0.01,lmd)
        loss = logitloss(sigmoid(w_opt,X_test), y_test).mean(axis=0)
        error.append(loss)
    error = np.array(error).reshape(10,1)
    if i==1:
        error_sum = error
    else:
        error_sum = np.hstack((error_sum,error))
    print(error_sum)
    i += 1
print('Done!')     


/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


optimal loss:  26.54992165939248
optimal loss:  41.70435069424559
optimal loss:  44.001390289468105
optimal loss:  32.027334575676846
optimal loss:  42.37253557698297
optimal loss:  31.734910491201124
optimal loss:  43.60933282148682
optimal loss:  41.77293885605393
optimal loss:  48.23768532912316
optimal loss:  0.8727859904801794
[[16.58770666]
 [10.94745867]
 [12.26798859]
 [13.03775255]
 [14.07733039]
 [12.99261079]
 [12.62274288]
 [13.805076  ]
 [12.35283628]
 [10.73982546]]
optimal loss:  32.4687241372396
optimal loss:  40.53195016988851
optimal loss:  34.996822495319634
optimal loss:  30.709264470630327
optimal loss:  23.69551813039955
optimal loss:  32.5790295022225
optimal loss:  38.87831676770543
optimal loss:  26.541600627745737
optimal loss:  27.166106630736998
optimal loss:  1.016233161884004
[[16.58770666 18.23250276]
 [10.94745867 13.13485958]
 [12.26798859 13.77732841]
 [13.03775255 13.66347995]
 [14.07733039 15.26415215]
 [12.99261079 12.94292481]
 [12.62274288 14.1857

In [47]:
error_cv = pd.DataFrame(data=error_sum,columns = lambda_candi)
cv_sta = error_cv.describe()
mean = cv_sta.loc['mean']
std = cv_sta.loc['std']
print('mean:\n',mean)
print('standart deviation:\n',std)
cv_sta

mean:
 0.00      12.943133
0.01      13.856366
0.10      13.486654
1.00      13.493957
10.00     11.912445
100.00     9.566151
Name: mean, dtype: float64
standart deviation:
 0.00      1.668927
0.01      2.153096
0.10      1.689474
1.00      2.009842
10.00     1.896108
100.00    1.320369
Name: std, dtype: float64


,0.0,0.01,0.1,1.0,10.0,100.0
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,12.943133,13.856366,13.486654,13.493957,11.912445,9.566151
std,1.668927,2.153096,1.689474,2.009842,1.896108,1.320369
min,10.739825,9.935755,10.359330,9.180051,9.303928,8.413958
25%,12.289201,12.990909,12.454563,12.647936,10.566749,8.735594
50%,12.807677,13.720404,13.743128,13.556275,11.675127,9.074235
75%,13.613245,14.847639,14.414518,14.624780,12.988668,10.013228
max,16.587707,18.232503,16.257095,16.098789,15.139805,12.745849


# Different Regularizion Term $\lambda ||w||$


In [98]:
def gradient_L1reg(y,x,w,lmd):
    #loss = logitloss(sigmoid(w,x),y).reshape(y.shape[0],)
    reg = lmd*np.ones((1501,1))
    reg[0] = 0 # not regularize the bias term
    return np.add(reg,np.dot(np.diag(np.subtract(sigmoid(w,x).reshape(y.shape[0],),y)),x)).mean(axis=0).reshape((1501,1))

def gradientDescent_L1reg(y,w,x,eta,lmd):
    for i in range(1000):
        lossPre = logitloss(sigmoid(w,x),y).sum()
        w = np.subtract(w,np.multiply(eta,gradient_L1reg(y,x,w,lmd)))
        lossPost = logitloss(sigmoid(w,x),y).sum()
        if (lossPre-lossPost)/lossPre<0.01:
            print('optimal loss: ',lossPost)
            break
    return w

w_opt_l1 = gradientDescent_L1reg(train_y, w_init, train_x, 0.01, 0.1)

/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


optimal loss:  0.3076377121645377


# Stochastic Gradient descent with L2 regularization

In [212]:
def gradient_SGD(y,x,w,lmd):
    #loss = logitloss(sigmoid(w,x),y).reshape(y.shape[0],)
    reg = lmd*w
    reg[0] = 0 # not regularize the bias term
    gradient = ((sigmoid_SGD(w,x)-y)*x).reshape((((sigmoid_SGD(w,x)-y)*x).shape[0],1))+reg
    return gradient

def sigmoid_SGD(w,x):
    result = np.exp(np.dot(x,w))/(1+np.exp(np.dot(x,w)))
    return result

def SGD_L2(y,w,x,eta,lmd):
    row = np.arange(x.shape[0])
    np.random.shuffle(row)
    for i in range(1000):
        lossPre = logitloss(sigmoid(w,x),y).sum()
        for j in range(len(row)):
            w = w - eta*gradient_SGD(y[j],x[j],w,lmd)
        lossPost = logitloss(sigmoid(w,x),y).sum()
        if (lossPre-lossPost)/lossPre<0.01:
            print('optimal loss: ',lossPost)
            break
    return w
w_opt_SGD=SGD_L2(train_y, w_init, train_x, 0.01, 0.01)



/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


optimal loss:  12.44875083638241


# Weighted Linear Least Square  
We may face the problem of heteroskedasticity in regression, so we implement weighted least square.  

The mathematical form of weighted linear least square is minimizing $\sum_{i=1}^nw_i(y_i-x_i\beta)^2$.   

In matrix:
$$min\  (Y-X\beta)^TW(Y-X\beta)$$  

Here, we assume the weight matrix is $W^{-1}$, $W$ is diagonal matrix of error.  

Hence, beta would be:
$$\beta^* = (X^TWX)^{-1}X^TWY$$

Add a L2 regularization term:
$$min\  (Y-X\beta)^TW(Y-X\beta) + \frac{\lambda}{2} ||\beta||^2$$  
$$\beta^* = (X^TWX+\lambda I)^{-1}X^TWY$$

In [26]:
import time

def WLS(X,y):
    b_ols = (np.dot(X.T,X).T).dot(np.dot(X.T,y))
    error = np.square(y - np.dot(X,b_ols))
    weight = np.diag(np.divide(1,error).reshape(error.shape[0]))
    b_wls = np.linalg.inv(np.dot(X.T,np.dot(weight,X))).dot(np.dot(X.T,np.dot(weight,y)))
    return np.array(b_wls)
start = time.time()
b_wls = WLS(train_x,train_y)
end = time.time()
print('time spent:',end-start)

def WLS_L2reg(X,y,lmd):
    b_ols = (np.dot(X.T,X).T).dot(np.dot(X.T,y))
    error = np.square(y - np.dot(X,b_ols))
    weight = np.diag(np.divide(1,error).reshape(error.shape[0]))
    b_wls = np.linalg.inv(np.add(np.dot(X.T,np.dot(weight,X)),lmd*np.ones((X.shape[1],X.shape[1])))).dot(np.dot(X.T,np.dot(weight,y)))
    return np.array(b_wls)

start = time.time()
b_wls = WLS_L2reg(train_x,train_y,0.1)
end = time.time()
print('time spent:',end-start)

def sqloss(y_,y):
    return np.mean(np.square(np.subtract(y_,y)))

# cross validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
lambda_candi = [0,0.01,0.1,1,10,100]
error_sum = []
i = 1
for lmd in lambda_candi:
    error = []
    for train_index, test_index in kf.split(train_x):
        w_init = np.random.rand(1501,1)
        X_train, y_train = train_x[train_index], train_y[train_index]
        X_test, y_test = train_x[test_index], train_y[test_index]
        w_opt = WLS_L2reg(X_train,y_train,lmd)
        loss = sqloss(np.dot(X_test,w_opt), y_test)
        error.append(loss)
    error = np.array(error).reshape(10,1)
    if i==1:
        error_sum = error
    else:
        error_sum = np.hstack((error_sum,error))
    print(error_sum)
    i += 1
print('Done!')   
  



time spent: 1.3622729778289795
time spent: 1.5403039455413818
[[ 25.8992379 ]
 [184.39661351]
 [  2.49887839]
 [  4.41977486]
 [  8.20673336]
 [ 12.75732468]
 [  3.06004279]
 [  2.11111552]
 [  6.37858705]
 [  1.84239537]]
[[ 25.8992379   21.78042674]
 [184.39661351  53.73338445]
 [  2.49887839   2.60874292]
 [  4.41977486   3.94462904]
 [  8.20673336   8.1422325 ]
 [ 12.75732468  10.77467758]
 [  3.06004279   2.94131683]
 [  2.11111552   1.84457153]
 [  6.37858705   6.77942798]
 [  1.84239537   2.33273667]]


KeyboardInterrupt: 

# Process Testset

numpy.ndarray

In [57]:
test_x = test.sms.values
test_y = test.label.values
test_token = vectorizer.transform(test_x)
test_token_idf = transformer.transform(test_token)
df_test_token = pd.DataFrame(data=test_token_idf.toarray(),columns=colname)
standard_df_test_token = standardScaler.transform(df_test_token)
df_test_token_pca=pca.transform(standard_df_test_token)
test_x = np.hstack((np.ones((test_y.shape[0],1)),df_test_token_pca))

# Calculate Test Error